This notebook generates a list of all the unique incidents tracked in the SIT 209 report
unique_fires.csv contains:
1. Incident Number	
2. Fire Name	
3. First Day
4. Last Day	
5. Lat Fire	
6. Lon Fire

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
np.set_printoptions(threshold=33500)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta

In [3]:
#load in all the data
sit_all =pd.read_csv('../merged_sit.csv', dtype={'INCIDENT_JURISDICTION': 'str', 'UNIT_OR_OTHER_NARR': 'str'})
#viirs_all = pd.read_csv('merged_viirs.csv', dtype={'satellite':'str', 'version': 'str'})

In [4]:
sit_all

,ID,INC209R_IDENTIFIER,INC_IDENTIFIER,DONWCGU_PROT_UNIT_IDENTIFIER,REPORT_FROM_DATE,REPORT_TO_DATE,STATUS,INCIDENT_DESCRIPTION,INCIDENT_COMMANDERS_NARR,CURR_INCIDENT_AREA,...,SECNDRY_FUEL_MODEL_IDENTIFIER,ADDNTL_FUEL_MODEL_IDENTIFIER,GEN_FIRE_BEHAVIOR_IDENTIFIER,FIRE_BEHAVIOR_1_IDENTIFIER,FIRE_BEHAVIOR_2_IDENTIFIER,FIRE_BEHAVIOR_3_IDENTIFIER,ROAD_CLOSURE_FLAG,AREA_CLOSURE_FLAG,TRAIL_CLOSURE_FLAG,UNIFIED_COMMAND_FLAG
0,7,10671660,10671438,9338017,2019-07-21 17:00:00,2019-07-22 17:00:00,U,NaN,Alan Sinclair SW-IMT1,7512.0,...,9352037.0,9352036.0,9352421.0,NaN,NaN,NaN,Y,Y,Y,N
1,8,10671703,10671438,9338017,2019-07-23 17:00:00,2019-07-24 17:00:00,U,NaN,Alan Sinclair SW-IMT1,7512.0,...,9352037.0,9352036.0,9352421.0,NaN,NaN,NaN,Y,Y,Y,N
2,9,10672949,10672775,9338294,2019-07-13 20:00:00,2019-07-14 20:00:00,I,NaN,ICT3 Steven Staples\nICT3 (t) Jon Rupp,600.0,...,9352042.0,9352037.0,9352420.0,9352424.0,9352433.0,9352429.0,Y,N,Y,N
3,10,10672964,10672775,9338294,2019-07-14 22:00:00,2019-07-15 19:00:00,U,NaN,Steven Staples & Jon Rupp (t),600.0,...,9352042.0,9352037.0,9352420.0,9352424.0,9352433.0,9352429.0,Y,N,Y,N
4,11,10672986,10672775,9338294,2019-08-08 17:00:00,2019-08-16 16:45:00,F,NaN,DUTY OFFICER - JON RUPP,5020.0,...,9352042.0,9352037.0,9352421.0,NaN,NaN,NaN,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13252,10815,11980634,11980587,9415365,2020-12-25 20:00:00,2020-12-26 07:45:00,U,NaN,"RUSHING,R. (CA-MCP) JOHNSON,R (CA-MVU)",4276.0,...,9429764.0,9429765.0,9430150.0,NaN,NaN,NaN,N,N,N,Y
13253,10816,11980711,11980587,9415365,2020-12-26 08:00:00,2020-12-26 19:00:00,U,NaN,"RUSHING,R. (CA-MCP)",4276.0,...,9429764.0,9429765.0,9430150.0,NaN,NaN,NaN,N,N,N,Y
13254,10817,11980598,11980587,9415365,2020-12-27 08:00:00,2020-12-27 19:00:00,U,NaN,"RUSHING,R. (CA-MCP)",4276.0,...,9429764.0,9429765.0,9430150.0,NaN,NaN,NaN,N,N,N,N
13255,10818,11980625,11980587,9415365,2020-12-28 08:00:00,2020-12-28 17:45:00,F,NaN,"RUSHING,R. (CA-MCP)",4276.0,...,9429764.0,9429765.0,9430150.0,NaN,NaN,NaN,N,N,N,N


In [5]:
#generate the wildfire and complex codes
wildfire_codes = []
complex_codes = []
name_lookup ='SIT209_HISTORY_SIT209_LOOKUP_CODES_*.xlsx'
for year in range(2019, 2021):
    df_lookup = pd.read_excel(name_lookup.replace('*', str(year)))
    print(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values)
    wildfire_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values[0])
    complex_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Complex']['LUCODES_IDENTIFIER'].values[0])
print(wildfire_codes, complex_codes)

#select the wildfires and complexes only
sit_fire = sit_all[(sit_all['INCTYP_IDENTIFIER']==wildfire_codes[0])|(sit_all['INCTYP_IDENTIFIER']==wildfire_codes[1])|(sit_all['INCTYP_IDENTIFIER']==complex_codes[0])|(sit_all['INCTYP_IDENTIFIER']==complex_codes[1])]
print('We have ' + str(len(sit_fire['INCTYP_IDENTIFIER'])) + ' wildfire and complex reports')
#sanity check to see if we filtered out the right thing
aux = sit_all[(sit_all['INCTYP_IDENTIFIER']!=wildfire_codes[0])&(sit_all['INCTYP_IDENTIFIER']!=wildfire_codes[1])&(sit_all['INCTYP_IDENTIFIER']!=complex_codes[0])&(sit_all['INCTYP_IDENTIFIER']!=complex_codes[1])]
print(str(len(aux['INCTYP_IDENTIFIER']))+ ' Incidents were removed')

# find the unique incidents
inc_nums = sit_fire['INC_IDENTIFIER'].values # a numpy array of all the incidents
inc_names = sit_fire['INCIDENT_NAME'].values
inc_start_dates = sit_fire['DISCOVERY_DATE'].values

#compare number of unique incident numbers, names, and start dates
print('We have ' + str(len(np.unique(inc_nums))) + ' unique incident numbers, ' + str(len(np.unique(inc_names)))+ ' unique incident names, and ' + str(len(np.unique(inc_start_dates)))+ ' unique start dates')


[9352235]
[9429964]
[9352235, 9429964] [9352456, 9430185]
We have 13221 wildfire and complex reports
36 Incidents were removed
We have 1355 unique incident numbers, 1335 unique incident names, and 1577 unique start dates


In [6]:
# generate a dataframe of incidents that contains:
# the name, number, earliest discovery date as a day, end date of final report as a day

unique_incs = np.unique(inc_nums)
n_incs = len(unique_incs) #number of unique incidents
print(n_incs)
incidents = pd.DataFrame({'Incident Number': np.nan*np.zeros(n_incs), 
                          'Fire Name': np.nan*np.zeros(n_incs), 
                         'First Day': np.nan*np.zeros(n_incs),
                        'Last Day': np.nan*np.zeros(n_incs),
                        'Lat Fire': np.nan*np.zeros(n_incs), 
                     'Lon Fire': np.nan*np.zeros(n_incs)}) # pre-allocate the dataframe
for ii in range(n_incs):
    #grab the name and incident number
    thisinc_num = unique_incs[ii] # Incident Number
    df_fire = sit_fire[sit_fire['INC_IDENTIFIER']==thisinc_num]
    df_fire = df_fire.sort_values('REPORT_FROM_DATE')
    thisinc_startdate = np.unique(df_fire['DISCOVERY_DATE'].values)
    thisinc_firstday = np.amin(thisinc_startdate)#[0:10] # First Day
    thisinc_enddate = df_fire['REPORT_TO_DATE'].values
    thisinc_lat = df_fire['POO_LATITUDE'].values[0]
    thisinc_lon = df_fire['POO_LONGITUDE'].values[0]
    if len(thisinc_enddate)>0:
        thisinc_lastday = np.amax(thisinc_enddate)[0:10] #get the last available day
    else:
        thisinc_lastday = np.nan
    
    #thisinc_lastday = thisinc_enddate[0:10] # Last Day
    thisinc_name = np.unique(df_fire['INCIDENT_NAME'].values) #Fire Name
    
    #fill in the whole row at once (format below)
    incidents.iloc[ii]= [thisinc_num, thisinc_name[0], thisinc_firstday, thisinc_lastday, thisinc_lat, thisinc_lon]
    
print(incidents)

1355
      Incident Number      Fire Name            First Day    Last Day  \
0          10662156.0            C34  2019-02-13 15:00:00  2019-02-15   
1          10662179.0           2161  2019-02-13 16:40:00  2019-02-14   
2          10662684.0  PAINTED WAGON  2019-04-08 13:02:00  2019-04-10   
3          10663171.0  LONE MOUNTAIN  2019-05-26 23:56:00  2019-07-03   
4          10663525.0        BOULDER  2019-06-05 12:45:00  2019-06-06   
...               ...            ...                  ...         ...   
1350       11979081.0        LAURA 2  2020-11-17 13:30:00  2020-11-23   
1351       11979237.0          Trail  2020-10-24 21:32:00  2020-11-02   
1352       11980186.0      SANDERSON  2020-12-13 03:00:00  2020-12-14   
1353       11980587.0        CREEK 5  2020-12-24 01:22:00  2020-12-28   
1354       11980949.0        CREEK 5  2020-12-24 01:00:00  2020-12-24   

       Lat Fire    Lon Fire  
0     36.995278 -102.524722  
1     35.221389 -101.518611  
2     33.786944 -112.753333 

In [7]:
#check for nan values of the fire location, and drop them!
te = incidents['Lat Fire'].values
drops = np.where(np.isnan(te))[0]
incidents = incidents.drop(labels=drops)

In [8]:
# filter out fires whose starting location is outside of our domain
incidents = incidents[(incidents['Lat Fire']>=31)&(incidents['Lat Fire']<=49)&(incidents['Lon Fire']>=-125)&(incidents['Lon Fire']<=-101)]
print(incidents)

      Incident Number      Fire Name            First Day    Last Day  \
0          10662156.0            C34  2019-02-13 15:00:00  2019-02-15   
1          10662179.0           2161  2019-02-13 16:40:00  2019-02-14   
2          10662684.0  PAINTED WAGON  2019-04-08 13:02:00  2019-04-10   
3          10663171.0  LONE MOUNTAIN  2019-05-26 23:56:00  2019-07-03   
4          10663525.0        BOULDER  2019-06-05 12:45:00  2019-06-06   
...               ...            ...                  ...         ...   
1350       11979081.0        LAURA 2  2020-11-17 13:30:00  2020-11-23   
1351       11979237.0          Trail  2020-10-24 21:32:00  2020-11-02   
1352       11980186.0      SANDERSON  2020-12-13 03:00:00  2020-12-14   
1353       11980587.0        CREEK 5  2020-12-24 01:22:00  2020-12-28   
1354       11980949.0        CREEK 5  2020-12-24 01:00:00  2020-12-24   

       Lat Fire    Lon Fire  
0     36.995278 -102.524722  
1     35.221389 -101.518611  
2     33.786944 -112.753333  
3  

In [9]:
# save incidents as a csv file
incidents.to_csv('unique_fires.csv', index=False)